## ライブラリのimport

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


## oofのdfセッティング

In [9]:
oof_csv_list = [
    "/root/signate_tecno/input/oof_CV0.9998_v2.csv",
    "/root/signate_tecno/input/oof_CV0.9995.csv",
    "/root/signate_tecno/input/oof_CV0.9994_1027.csv",
    "/root/signate_tecno/input/oof_CV0.9994_1030.csv",
]
sub_csv_list = [
    "/root/signate_tecno/input/submission_CV_best.csv",
    "/root/signate_tecno/input/submission_CV_lstm_v1.csv",
    "/root/signate_tecno/input/submission_CV_effnet_lstm_1030_cutmix.csv",
    "/root/signate_tecno/input/submission_CV_effnet_lstm_[1030_not_cutmix].csv"


]

In [ ]:
pd.read_csv("/root/signate_tecno/input/submission_CV_best.csv",header = None)

In [ ]:
# Initialize an empty DataFrame
stacking_df = pd.DataFrame()
temp_df = pd.read_csv("/root/signate_tecno/input/oof_CV0.9998_v2.csv")
stacking_df['label'] = temp_df['label']
# Loop through each CSV file in the list
for i, csv_path in enumerate(oof_csv_list):
    # Read the CSV file
    temp_df = pd.read_csv(csv_path)
    
    # Add the 'oof' column to the stacking_df with a new column name
    stacking_df[f'oof_{i}'] = temp_df['oof']

# Display the resulting DataFrame
print(stacking_df.head())

stacking_test_df = pd.DataFrame()

for i, csv_path in enumerate(sub_csv_list):
    temp_df = pd.read_csv(csv_path,header = None)
    temp_df.columns = ['file_name', 'label', 'oof']
    stacking_test_df[f'oof_{i}'] = temp_df['oof']

In [ ]:
stacking_test_df

In [ ]:
stacking_test_df

In [41]:
# ====================================================
# Configurations
# ====================================================
class CFG:
  VER = 1.0
  AUTHOR = 'shimizu'
  COMPETITION = 'stacking'
  exp = "001"
  METHOD_LIST = ['logistic_regression']
  seed = 42
  n_folds = 5
  target_col = "label"
  metric = ""
  metric_maximize_flag = True
  num_boost_round = 1000
  early_stopping_round = 500
  verbose = 25
  classification_lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.1,  # 学習率を下げる
    'seed': seed,
    'verbose': -1,
    'is_unbalance': True,
    'boosting_type': 'gbdt',
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'max_depth':32,
    }
  classification_cat_params = {
            'learning_rate': 0.01,
            'iterations': num_boost_round,
            'random_seed': seed,
            'task_type': 'CPU',
            'eval_metric': 'AUC',
        }
  model_weight_dict = {'logistic_regression':1.0}


In [ ]:
stacking_test_df

In [ ]:
# Prepare the data
from sklearn.svm import SVC
y = stacking_df['label'].dropna().astype(int)  # Ensure labels are integers
X = stacking_df[["oof_1","oof_2", "oof_3","oof_0"]]

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)

# Initialize arrays to store results
oof_preds = np.zeros(X.shape[0])
models = []

# Stratified K-Fold Cross Validation
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"Fold {fold + 1}")
    
    # Split data
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]



# モデルの初期化
    model = SVC(probability=True, random_state=CFG.seed)
    model.fit(X_train, y_train)
    
    # Predict on validation set
    oof_preds[val_idx] = model.predict_proba(X_val)[:,1]

    # Save the model
    models.append(model)

# Calculate overall AUC
auc = roc_auc_score(y, oof_preds)
print(f"Overall AUC: {auc:.4f}")

In [56]:
def logistic_regression_inference(x_test:pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = models[fold]
        
        pred = model.predict_proba(x_test)[:,1]
        test_pred += pred
    return test_pred / CFG.n_folds

def model_inference(method:str,test_df:pd.DataFrame,features:list):
    x_test = test_df[features]
    if method == 'logistic_regression':
        test_pred = logistic_regression_inference(x_test)
    else:
        raise ValueError(f"Unknown method: {method}")
    return test_pred

def Predicting(input_df:pd.DataFrame, features:list):
    output_df = input_df.copy()
    output_df['pred_prob'] = 0
    for method in CFG.METHOD_LIST:
        output_df[f'{method}_pred_prob'] = model_inference(method,input_df,features)
        output_df['pred_prob'] += CFG.model_weight_dict[method]* output_df[f'{method}_pred_prob']
    return output_df

In [57]:
test_df = Predicting(stacking_test_df,["oof_1","oof_2", "oof_3","oof_0"])

In [ ]:
test_df

In [ ]:
# Concatenate submit_df and test_df along columns
merged_df = pd.concat([test_df, submit], axis=1)
merged_df.tail()

In [ ]:
# Create a DataFrame with binary values based on the condition
binary_df = (merged_df[['oof_0', 'oof_1', 'oof_2', 'oof_3','image_path']])
binary_df =(binary_df[['oof_0', 'oof_1', 'oof_2', 'oof_3']] > 0.5).astype(int)
# Find rows where there is a disagreement among the columns
disagreement_rows = binary_df[binary_df.nunique(axis=1) > 1]



In [ ]:
# Create a DataFrame with binary values based on the condition
binary_df = merged_df[['oof_0', 'oof_1', 'oof_2', 'oof_3', 'image_path']]

# 'oof'列をバイナリ化
binary_df[['oof_0', 'oof_1', 'oof_2', 'oof_3']] = (binary_df[['oof_0', 'oof_1', 'oof_2', 'oof_3']] > 0.5).astype(int)

# 'oof'列において意見が異なる行を抽出
disagreement_rows = binary_df[binary_df[['oof_0', 'oof_1', 'oof_2', 'oof_3']].nunique(axis=1) > 1]

# 該当するimage_pathを取得
image_paths = disagreement_rows['image_path']

# 必要に応じて結果を表示



In [88]:
image_paths.to_csv("/root/signate_tecno/submit/disagreement_indices.csv",header = None)

In [59]:
submit = pd.read_csv(f"/root/signate_tecno/input/sample_submit.csv", header=None, names=['image_path', 'label'])

In [ ]:
submit.shape

In [ ]:
test_df.shape

In [ ]:
submit

In [ ]:
submit['label'] = (test_df["pred_prob"] > 0.5).astype(int)
submit['label'].value_counts()

In [ ]:
submit

In [61]:
submit['label'] = submit['label'].apply(lambda x: 0 if x == 1.0 else 1)

In [65]:
submit.to_csv(f'/root/signate_tecno/input/submission_stacking_svc[1030].csv',index = False, header = None)
